### Parse Komoot tracks

In [17]:
import gpxpy
from shapely.geometry import LineString
from swak.funcflow import Pipe
import psycopg2

In [52]:
# Load and parse GPX
def load_gpx(path):
    with open(path, "r") as f:
        return gpxpy.parse(f)


def extract_linestring(gpx: gpxpy.gpx.GPX):
    points = []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                if point.elevation is not None:
                    points.append((point.longitude, point.latitude, point.elevation))
    return LineString(points) 


In [53]:
l = Pipe(load_gpx, 
          extract_linestring, 
          # The linestring object is really cool and allows for all kind of things
          # Here we just want to put the wkt (well known text) into the postgres
          lambda l: l.wkt,
         )("../../resources/gps/schwemmkanalweg_gross_komoot.gpx")

In [55]:
t = "01020000A0E6100000FD0000000C40A374E90F2C402EE411DC484F484082035ABA62C98440111AC1C6F50F2C40E3E2A8DC444F484082035ABA62C984403F027FF8F90F2C40B492567C434F484082035ABA62C984406440F67AF70F2C40B5C6A013424F484082035ABA62C98440A25F5B3FFD0F2C40F0F8F6AE414F484082035ABA62C984400B7A6F0C01102C4048DC63E9434F484082035ABA62C984408BDF14562A102C40CCEEC9C3424F484082035ABA62C98440B83B6BB75D102C4003CC7C073F4F484082035ABA62C98440D5264EEE77102C404BAC8C463E4F48404F779E784ECA84406876DD5B91102C40CE8AA8893E4F4840B5A84F7247CC84406893C3279D102C40234DBC033C4F4840BCB376DBC5CD8440EBC891CEC0102C40E29178793A4F4840EA094B3CA0D08440C5AC174339112C40BFEFDFBC384F4840682096CDFCD98440B9E00CFE7E112C40A20A7F86374F4840B08EE3876ADF8440A27DACE0B7112C40797764AC364F4840D481ACA7D6E384405D4F745DF8112C4042CEFBFF384F4840DA8F149121D684406E179AEB34122C403620425C394F4840AD68739C7BC48440D40FEA2285122C4007D0EFFB374F4840E6CC7685FEAC844045F6419605132C403FE1ECD6324F484096AFCBF07F868440C90050C58D132C4029ED0DBE304F4840ED29392736648440197442E8A0132C407099D365314F4840CBBF96578E5F84400B47904AB1132C4081D07AF8324F4840A9DE1AD80A5B8440DF3312A111142C4021E527D53E4F48406E5166836C3E8440401878EE3D142C409C6A2DCC424F484010AFEB176C328440CBF78C4468142C40F581E49D434F48404FE5B4A7442884406DAE9AE788142C40F7E978CC404F484025CCB4FD8B2584409143C4CDA9142C40F2608BDD3E4F4840C156091607238440AD68739CDB142C404BAC8C463E4F48408A0453CD6C1F844082380F2730152C40A4C343183F4F484094162EAB50198440F9F36DC152152C406308008E3D4F48405551BCCABA168440F0A5F0A0D9152C40A47213B5344F4840F0F9618490048440923F1878EE152C404B5B5CE3334F4840C0594A9693FD83400C3F389F3A162C40EB5223F4334F4840B5FE9600BCE48340FAD2DB9F8B162C40C4279D48304F484053094FE855C98340E9D500A5A1162C408EE6C8CA2F4F4840425DA45016C2834096E99788B7162C40CFA10C55314F4840664F029B53BA8340BE310400C7162C40FB213658384F48402C7FBE2D98AB8340D74CBED9E6162C400DDE57E5424F48403A4030472F98834010E9B7AF03172C40A67C08AA464F4840A2B5A2CD31948340BB0F406A13172C40236AA2CF474F4840834E081D5492834084D4EDEC2B172C409C8713984E4F4840753BFBCA238D83406EE00ED429172C401EFE9AAC514F48402BA4FCA41A8B8340B58CD47B2A172C4059FCA6B0524F484032AA0CE36E8A8340AFEC82C135172C40E5D53906644F4840B58B69A6FB7E834026A8E15B58172C40CEFE40B96D4F4840B1F7E28B96778340B14D2A1A6B172C406C43C5387F4F48401B683EE7AE8A834089963C9E96172C408B4F01309E4F48407DCA31599CCB834034DAAA24B2172C40F299EC9FA74F484058703FE001E183402E1D739EB1172C4079CDAB3AAB4F4840F8AA95095FE88340CBBF9657AE172C402F8507CDAE4F4840A019C407D6F18340CD72D9E89C172C400A1346B3B24F48404C50C3B7F0008440F833BC5983172C409146054EB64F4840B1A547533D128440E7FC14C781172C4043A9BD88B64F4840D95C35CF51138440460C3B8C49172C4094331477BC4F4840D73043E3A935844015FDA19927172C400D1D3BA8C44F4840EF737CB4985484401B9DF3531C172C4098A59D9ACB4F484008556AF6006B8440DD9A745B22172C4071CB4752D24F4840751DAA2929808440511212691B172C40AEB6627FD94F48409E077767CD968440CC63CDC820172C40A629029CDE4F484062BCE6553DA684404B8FA67A32172C40AF0793E2E34F48404B5B5CE333B5844067D13B1570172C40787B1002F24F48402C9CA4F963E084400135B56CAD172C40419E5DBEF54F484033C2DB8330FB84402B8716D9CE172C4074435376FA4F4840CE5147C7350D8540EFE53E390A182C4029B000A60C5048407077D66EDB388540A839799109182C40030AF5F4115048403160C955CC3C8540226E4E2503182C4095B7239C16504840F9A3A83357408540759483D904182C40ACAB02B51850484055DFF945E94185407A8B87F71C182C40F4A8F8BF235048402788BA0FA04A854021567F8461182C40FED7B969335048401A6D5512F958854046EBA86A82182C405B785E2A36504840E141B3EB7E5D85409ECE15A584182C404FCAA486365048400A2E56D4E05D8540B22C98F8A3182C401A89D00836504840437232718B618540B2A03028D3182C40376E313F3750484092921E867664854079B29B19FD182C4060CD018239504840187B2FBE28678540FB95CE8767192C40D15D126745504840936E4BE4026F8540FB95CE8767192C401381EA1F445048409E95B4E27B6F8540FB95CE8767192C40D15D126745504840A9BC1DE1F46F85405396218E75192C40C93846B24750484084D6C39729718540B340BB438A192C4055C1A8A44E504840C806D2C506748540A8C64B3789192C40043DD4B66150484064575A460A8185404A26A77686192C40E353008C675048403A02B8597C8685403602F1BA7E192C40BE1589096A5048407D5BB05417898540643C4A253C192C4012F5824F73504840D4635B067C96854076C58CF0F6182C408F334DD87E50484062D8614C9AA5854004AA7F10C9182C40F549EEB089504840E8BF07AF9DB1854013D731AEB8182C404A29E8F6925048405E68AED3E8B9854054C6BFCFB8182C407DCEDDAE975048403FC571E0F5BD8540D7A4DB12B9182C40A25D85949F504840ABCE6A81BDC485402C9B3924B5182C40B22C98F8A3504840B6D617098DC8854060066344A2182C40910F7A36AB5048408177F2E951CF854071AC8BDB68182C404ED53DB2B9504840F20698F92EDE85408E06F01648182C404B395FECBD5048401A6B7F675BE38540B000A60C1C182C407C0E2C47C8504840D4B5F63EF5EB854064062AE3DF172C40374F75C8CD5048401F80D426AEF38540F20703CFBD172C40095053CBD6504840998235CEE6FA8540616BB6F292172C40B4AA251DE5504840F390291FC205864029ECA2E881172C4003CDE7DCED504840CB4752D2E30E86409206B7B585172C4048A98427F450484074ED0BE8E5168640D6FF39CC97172C402C499EEBFB504840454772F96F2186408109DCBA9B172C40E8BD310400514840D7DEA7AAB02686408109DCBA9B172C404C1B0E4B03514840D21BEE23D72A8640A33D5E4887172C404F594DD713514840F8E0B54B3B40864065C746205E172C4088F546AD305148402B357BA0F57286402F52280B5F172C400EF5BBB035514840E1D05B3CDC7D8640FBC9181F66172C40FF76D9AF3B514840471D1D57238B8640E40F069E7B172C4072A3C85A43514840070951BE809D86400FD6FF39CC172C40170D198F52514840B62FA0176EC98640F568AA27F3172C4032569BFF575148400307B474E5D88640CAC4AD8218182C40789ACC785B51484026732CEF0AE3864063EE5A423E182C40717500C45D514840ADBEBA2A90EC86404C8BFA2477182C40508C2C9963514840EC67B11469FC8640325871AAB5182C40DD7C23BA6751484075779D0D590C8740E5284014CC182C40593673486A514840F661BD51CB12874040F9BB77D4182C40B6D617096D514840E86C01A135178740FF092E56D4182C40FC1A4982705148407C7BD7A02F1C874007978E39CF182C408A73D4D17151484052B81E856B1E8740A795422097182C402D246074795148402079E750662E87403B8E1F2A8D182C409772BED87B514840306475ABE7318740B5C2F4BD86182C40A741D13C80514840029CDEC55B3787409BFEEC478A182C40DFA3FE7A855148409B58E02BBA3D8740C07630629F182C4072512D228A51484098A25C1ABF448740C9207711A6182C405E4A5D328E514840B345D26ED449874062D68BA19C182C40C9E9EBF99A514840145E82535F598740B5FCC0559E182C403C16DBA4A25148407AE063B0A262874013807F4A95182C402C64AE0CAA5148406C41EF8DC16B87408AE42B8194182C40CBF3E0EEAC514840CFD72C974D6F8740433866D993182C40B2632310AF51484055DFF945897287408461C092AB182C400FED6305BF514840BC02D193728B8740D7A4DB12B9182C4029EB3713D351484043E7357609AA8740C95A43A9BD182C4015014EEFE25148408E03AF961BC28740CE177B2FBE182C4073A1F2AFE5514840488C9E5B48C687407381CB63CD182C405166834C325248405CC98E8DA01D884070EB6E9EEA182C40081D74098752484055C2137A7D7388404303B16CE6182C407C7DAD4B8D524840310BED9C867988403562669FC7182C40265305A392524840C4EDD0B098808840FCE25295B6182C409BE7887C97524840E962D34AC184884059DAA9B9DC182C408E05854199524840BC033C6901888840FCE25295B6182C409BE7887C975248404DDC2A88418B884032C7F2AE7A182C40B4AB90F29352484036936FB679908840C93846B247182C4081069B3A8F524840DE57E542459588403C4F3C670B182C40CD1E68058652484018CDCAF6019C884009707A17EF172C40BFB7E9CF7E52484025EA059F46A08840C2A6CEA3E2172C40CD0182397A5248402046088FD69E884061191BBAD9172C409B90D6187452484000E14389F69B8840BF9CD9AED0172C4083177D056952484030134548DD968840E4BD6A65C2172C408192020B6052484085E97B0DA192884031ED9BFBAB172C40FD2E6CCD56524840211FF46C168E88403D0B42791F172C4037514B732B52484070404B5790688840BBF2599E07172C40132A38BC20524840CEFC6A0ED05D88408D96033DD4162C4070D1C952EB514840202922C34A2C884090662C9ACE162C407DCA3159DC514840069CA564B91E8840DA73999A04172C40C1ADBB79AA514840F75AD07BA3DF87409BC8CC052E172C40758F6CAE9A514840AC5626FC92C78740658D7A8846172C40B03C484F91514840A19FA9D76DB38740B4006DAB59172C40F10EF0A485514840B345D26E949B87409B3C65355D172C4005C58F3177514840A79201A00A7F87401B118C834B172C40CEFE40B96D514840755B22177C6B87401EC4CE143A172C401E6CB1DB67514840304B3B35975E87404FEB36A8FD162C404C6BD3D85E5148401DAF40F484488740A19DD32CD0162C4068976F7D58514840952A51F6563987402D095053CB162C40506F46CD57514840BC5983F7B53787408F006E162F162C404E999B6F44514840A41B6151F10587401288D7F50B162C406F826F9A3E51484004CAA65CA1FC86409A0645F300162C4035ECF7C43A514840C3B81B442BFB864073DBBE47FD152C40DF701FB935514840DFF8DA336BF986403C8386FE09162C404B3E761728514840CBDB114EABF486409A40118B18162C4055336B29205148407AFB73D1D0F186401A4F04711E162C406249B9FB1C514840530438BDABF08640B9DE365321162C4078B81D1A1651484093A7ACA64BEE8640834C327216162C4045132862115148408D4127848EEC8640F6622827DA152C40C93CF20703514840715AF0A20FEA86407C9DD497A5152C400F643DB5FA5048403E3C4B9031EB864046D1031F83152C40996725ADF85048406B80D250C3EB8640492D944C4E152C402828452BF7504840782975C998EC864044A51133FB142C40FA0B3D62F450484095271076EAED86407AFCDEA63F132C400B43E4F4F5504840BEC1172633FE8640C5E74EB0FF122C4047753A90F5504840452BF702930387405F07CE1951122C40F261F6B2ED504840126745D4C41287403B72A43330122C40FE43FAEDEB5048404AEEB089AC158740FCDEA63FFB112C406DCA15DEE55048406B2A8BC2AE1B87402A8D98D9E7112C403FE257ACE1504840D845D103BF1F874060E5D022DB112C407E18213CDA50484072A43330B2258740F10D85CFD6112C40669FC728CF504840569BFF573D2E874066A208A9DB112C4081971936CA5048408FDE701F1932874019E76F4221122C403815A930B6504840EFAEB321DF4387400D8AE6012C122C407D08AA46AF504840D2FE07580B478740A75CE15D2E122C404776A565A4504840A96741280F468740C9AD49B725122C4027A435069D5048409031772D61458740F1643733FA112C4046CEC29E7650484063B7CF2A734087405A626534F2112C406C4084B8725048409775FF58683B8740B6BE4868CB112C40172D40DB6A5048401C446B45FB2E87407DE882FA96112C40EBE0606F6250484035423F532F20874058703FE081112C402F6CCD565E504840DA56B3CE98198740FD9FC37C79112C40EEB089CC5C50484086048C2E0F1787407FDE54A4C2102C403E93FDF334504840E1270EA0BFD386408A3BDEE4B7102C404CA94BC6315048409CF86A4771CE86408D0B0742B2102C4060B01BB62D5048407D09151C5EC8864098689082A7102C408048BF7D1D504840EEE87FB996B08640637FD93D79102C40B56CAD2F1250484073BD6DA6829E8640051901158E102C40A301BC05125048404487C091C09A8640BAF3C473B6102C40DC4603780B50484098FC4FFE8E908640EEB5A0F7C6102C40B22E6EA30150484060AE450B708586402C0FD253E4102C4009C1AA7AF94F484074ECA012177B86400CC85EEFFE102C407022FAB5F54F484099F56228C7748640BFB5132521112C4084F57F0EF34F484084D72E6DF86D8640861E317A6E112C40FDC1C073EF4F484041834D9D275F86401D21037976112C4038F4160FEF4F484079909E22875D8640B340BB438A112C4069ACFD9DED4F48405DA3E5404F59864014083BC5AA112C40FF91E9D0E94F4840FC389A238B5186405BD1E638B7112C4066BFEE74E74F4840EC34D252D94D8640D252793BC2112C40BBB54C86E34F4840FC372F4EBC488640E38920CEC3112C404030478FDF4F484056293DD30B4486406FF59CF4BE112C408978EBFCDB4F48403CDD79E2B93F86403C50A73CBA112C4007CE1951DA4F48407BBE66B98C3D864063416150A6112C40037976F9D64F48400953944B03388640DEC7D11C59112C404F9143C4CD4F48401004C8D0712586400C59DDEA39112C40B29DEFA7C64F48404BAE62F1DB17864070253B3602112C403221E692AA4F48405C7171548ECF85406A4B1DE4F5102C407B4CA4349B4F48403B8BDEA9A0A98540FE261422E0102C405B7A34D5934F484078978BF88E9585407651F4C0C7102C4000AAB8718B4F4840240A2DEB1E848540849B8C2AC3102C407B12D89C834F4840588FFB56AB7785409B8F6B43C5102C408EB1135E824F484027158DB59F758540D7DEA7AAD0102C4026FF93BF7B4F4840DDEC0F94BB6A8540EA3C2AFEEF102C40CB2E185C734F48402F36AD14225B854026A94C3107112C40DAACFA5C6D4F4840C58EC6A1DE4F8540454772F90F112C40B14D2A1A6B4F48408B4F01309E4B8540D6C6D8092F112C40BB421F2C634F48401B9E5E29AB3C8540EDD79DEE3C112C40FE99417C604F4840A4A65D4C133785409AEB34D252112C4023D8B8FE5D4F484028452BF7423085408CF84ECC7A112C402AC93A1C5D4F48405F9A22C0A9288540D3DEE00B93112C40FA449E245D4F4840ECDB4944182485401C959BA8A5112C407E5704FF5B4F4840C90391455A208540CF488446B0112C40C0AE264F594F48408F71C5C5B11C854017F549EEB0112C40562C7E53584F4840C9570229911B8540DBC2F352B1112C400F80B8AB574F4840DB4FC6F8D01A8540CD5B751DAA112C409F7422C1544F4840336E6AA03917854052EDD3F198112C40F9F36DC1524F4840828E56B54413854064B0E2546B112C407E3A1E33504F48400C06D7DC310A8540A3AD4A22FB102C40ECC039234A4F48407A3881E9F4F3844081221631EC102C409966BAD7494F484051DCF1261FF18440849B8C2AC3102C40404F0306494F48409DBB5D2F6DED8440467C2766BD102C40726F7EC3444F48409DBB5D2F6DED8440B5368DEDB5102C40198C1189424F48409DBB5D2F6DED84406876DD5B91102C40CE8AA8893E4F48409DBB5D2F6DED8440D5264EEE77102C404BAC8C463E4F48409DBB5D2F6DED8440B83B6BB75D102C4003CC7C073F4F48409DBB5D2F6DED84408BDF14562A102C40CCEEC9C3424F48409DBB5D2F6DED84400B7A6F0C01102C4048DC63E9434F48409DBB5D2F6DED8440A25F5B3FFD0F2C40F0F8F6AE414F48409DBB5D2F6DED84406440F67AF70F2C40B5C6A013424F48409DBB5D2F6DED84403F027FF8F90F2C40B492567C434F48409DBB5D2F6DED8440111AC1C6F50F2C40E3E2A8DC444F48409DBB5D2F6DED8440"

In [66]:
from shapely.geometry import mapping
from shapely import wkb
mapping(wkb.loads(t))['coordinates']

((14.031078, 48.619411, 665.173207),
 (14.031172, 48.619289, 665.173207),
 (14.031204, 48.619247, 665.173207),
 (14.031185, 48.619204, 665.173207),
 (14.031229, 48.619192, 665.173207),
 (14.031258, 48.61926, 665.173207),
 (14.031573, 48.619225, 665.173207),
 (14.031965, 48.619111, 665.173207),
 (14.032165, 48.619088, 665.288316),
 (14.032359, 48.619096, 665.534886),
 (14.032449, 48.619019, 665.72161),
 (14.032721, 48.618972, 666.07824),
 (14.03364, 48.618919, 667.248439),
 (14.034172, 48.618882, 667.927017),
 (14.034606, 48.618856, 668.479812),
 (14.035098, 48.618927, 666.76639),
 (14.03556, 48.618938, 664.560357),
 (14.036172, 48.618896, 661.624278),
 (14.037152, 48.618739, 656.812471),
 (14.038191, 48.618675, 652.526442),
 (14.038337, 48.618695, 651.944503),
 (14.038462, 48.618743, 651.380295),
 (14.039197, 48.619105, 647.802985),
 (14.039535, 48.619226, 646.30278),
 (14.039858, 48.619251, 645.033523),
 (14.040107, 48.619165, 644.693355),
 (14.040358, 48.619106, 644.37846),
 (14.0407

In [ ]:
### Insert into postgres

In [37]:
from geoalchemy2 import Geometry

In [39]:
Geometry(geometry_type="LINESTRINGZ", srid=4326)

Geometry(geometry_type='LINESTRINGZ', srid=4326, dimension=3, from_text='ST_GeomFromEWKT', name='geometry')

In [ ]:
# Insert into PostGIS
conn = psycopg2.connect("dbname=yourdb user=youruser password=yourpass host=localhost")
cur = conn.cursor()
cur.execute("""
    INSERT INTO hiking_tracks (name, geom)
    VALUES (%s, ST_GeomFromText(%s, 4326))
""", ("Schwemmkanalweg_groß", wkt))
conn.commit()
cur.close()
conn.close()